In [ ]:
!pip install pyserini
!pip install faiss-cpu
!git clone https://github.com/castorini/anserini.git --recurse-submodules

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 130.5 MB 31 kB/s 
     |████████████████████████████████| 1.5 MB 34.9 MB/s 
     |████████████████████████████████| 2.0 MB 47.3 MB/s 
     |████████████████████████████████| 13.4 MB 1.4 MB/s 
     |████████████████████████████████| 5.8 MB 37.8 MB/s 
     |████████████████████████████████| 1.3 MB 13.9 MB/s 
     |████████████████████████████████| 4.5 MB 31.1 MB/s 
     |████████████████████████████████| 188 kB 27.4 MB/s 
     |████████████████████████████████| 46 kB 1.3 MB/s 
     |████████████████████████████████| 7.6 MB 2.7 MB/s 
     |████████████████████████████████| 182 kB 53.3 MB/s 
     |████████████████████████████████| 86 kB 2.8 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3
Looking in indexes: https://pypi.org/simple, https

In [ ]:
!wget https://trec.nist.gov/data/clinical/qrels-treceval-2016.txt
!wget http://ceb.nlm.nih.gov/~robertske/pmc-02.tar.gz  -P data/cds-corpus/
!wget http://ceb.nlm.nih.gov/~robertske/pmc-03.tar.gz  -P data/cds-corpus/

--2022-12-15 18:14:07--  https://trec.nist.gov/data/clinical/qrels-treceval-2016.txt
Resolving trec.nist.gov (trec.nist.gov)... 129.6.13.19, 2610:20:6005:13::19
Connecting to trec.nist.gov (trec.nist.gov)|129.6.13.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 553709 (541K) [text/plain]
Saving to: ‘qrels-treceval-2016.txt’

qrels-treceval-2016 100%[===================>] 540.73K  --.-KB/s    in 0.1s    

2022-12-15 18:14:07 (5.43 MB/s) - ‘qrels-treceval-2016.txt’ saved [553709/553709]

--2022-12-15 18:14:08--  http://ceb.nlm.nih.gov/~robertske/pmc-02.tar.gz
Resolving ceb.nlm.nih.gov (ceb.nlm.nih.gov)... 130.14.65.15, 2607:f220:41e:7065::15
Connecting to ceb.nlm.nih.gov (ceb.nlm.nih.gov)|130.14.65.15|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://ceb.nlm.nih.gov/~robertske/pmc-02.tar.gz [following]
--2022-12-15 18:14:08--  https://ceb.nlm.nih.gov/~robertske/pmc-02.tar.gz
Connecting to ceb.nlm.nih.gov (ceb.nl

In [ ]:
# Only extract certain files because of file size limitations on google colab


!ls data/cds-corpus/ 
# !tar -xvf "/content/data/cds-corpus/pmc-00.tar.gz" -C "data/cds-corpus"
# !tar -xvf "/content/data/cds-corpus/pmc-01.tar.gz" -C "data/cds-corpus"
!tar -xvf "/content/data/cds-corpus/pmc-02.tar.gz" -C "data/cds-corpus"
!tar -xvf "/content/data/cds-corpus/pmc-03.tar.gz" -C "data/cds-corpus"

Se han truncado las últimas 5000 líneas del flujo de salida.
pmc-03/71/4106166.nxml
pmc-03/71/4092004.nxml
pmc-03/71/4108467.nxml
pmc-03/71/4108469.nxml
pmc-03/71/4091551.nxml
pmc-03/71/4106167.nxml
pmc-03/71/4126836.nxml
pmc-03/71/4126073.nxml
pmc-03/71/4108464.nxml
pmc-03/71/4125380.nxml
pmc-03/71/4108468.nxml
pmc-03/71/4108457.nxml
pmc-03/71/4121338.nxml
pmc-03/71/4121337.nxml
pmc-03/71/4106164.nxml
pmc-03/71/4126837.nxml
pmc-03/71/4108459.nxml
pmc-03/71/4121339.nxml
pmc-03/71/4108466.nxml
pmc-03/71/4125379.nxml
pmc-03/71/4108458.nxml
pmc-03/71/4121340.nxml
pmc-03/71/4091053.nxml
pmc-03/71/4108460.nxml
pmc-03/71/4106160.nxml
pmc-03/71/4108462.nxml
pmc-03/71/4108463.nxml
pmc-03/71/4108461.nxml
pmc-03/71/4091098.nxml
pmc-03/71/3988120.nxml
pmc-03/71/3997517.nxml
pmc-03/71/4008453.nxml
pmc-03/71/4006859.nxml
pmc-03/71/3962508.nxml
pmc-03/71/3989296.nxml
pmc-03/71/4006854.nxml
pmc-03/71/3973659.nxml
pmc-03/71/4006851.nxml
pmc-03/71/3984267.nxml
pmc-03/71/4006852.nxml
pmc-03/71/4006856.n

In [ ]:
import json
import os
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET 
from bs4 import BeautifulSoup
import re

In [ ]:
with open('qrels-treceval-2016.txt', 'r') as f:
    data = f.readlines()

doc_set = set()
for line in data:
    doc_set.add(line.split(' ')[2])
print(len(doc_set))

28349


In [ ]:
paths = []

In [ ]:
# This is to get the directory that the program
# is currently running in.
for root, dirs, _ in os.walk('/content/data/cds-corpus/'):
    for dir in dirs:
        for _, subdirs, _ in os.walk(f'/content/data/cds-corpus/{dir}'):
            for subdir in subdirs:
                for _,_,files in os.walk(f'/content/data/cds-corpus/{dir}/{subdir}'):
                    for f in files:
                        if f.split('.')[0] in doc_set:
                            paths.append(f"{root}{dir}/{subdir}/{f}")

In [ ]:
test_path = paths[0]
print(test_path)
print(test_path.split('/')[6][:-5])

/content/data/cds-corpus/pmc-02/05/2565429.nxml
2565429


In [ ]:
with open('paths_qrels_02_03.json', 'w') as my_list_file:
  json.dump(paths, my_list_file)

In [ ]:
corpus_cds = {}

for path in paths: 
    f = open(path, 'r')
    soup = BeautifulSoup(f, 'html.parser')
    text = re.sub(r'http\S+', '', soup.get_text())
    doc = path.split('/')[6][:-5]
    corpus_cds[doc.split('.')[0]] = text
    f.close()

In [ ]:
!mkdir /content/data/new_document_parses

In [ ]:
with open('corpus_02_03.json', 'r') as my_list_file:
  data = json.load(my_list_file)

  for key in data.keys():
    corpus_cds = {}
    corpus_cds['id'] = key
    corpus_cds['contents'] = data[key]

    with open('/content/data/new_document_parses/{}.json'.format(key), 'w') as f:
      json.dump(corpus_cds, f)

len(os.listdir('/content/data/new_document_parses'))


16333

In [ ]:
!python -m pyserini.index.lucene -collection JsonCollection -generator DefaultLuceneDocumentGenerator -threads 9 \
-input /content/data/new_document_parses/ -index indexes/new_index -storePositions -storeDocvectors -storeRaw

Se han truncado las últimas 5000 líneas del flujo de salida.
2022-12-15 19:12:56,760 DEBUG [pool-2-thread-6] index.IndexCollection$LocalIndexerThread (IndexCollection.java:215) - new_document_parses/3005645.json: 1 docs added.
2022-12-15 19:12:56,763 DEBUG [pool-2-thread-6] index.IndexCollection$LocalIndexerThread (IndexCollection.java:215) - new_document_parses/3276037.json: 1 docs added.
2022-12-15 19:12:56,764 DEBUG [pool-2-thread-5] index.IndexCollection$LocalIndexerThread (IndexCollection.java:215) - new_document_parses/3193663.json: 1 docs added.
2022-12-15 19:12:56,775 DEBUG [pool-2-thread-2] index.IndexCollection$LocalIndexerThread (IndexCollection.java:215) - new_document_parses/2808924.json: 1 docs added.
2022-12-15 19:12:56,769 DEBUG [pool-2-thread-9] index.IndexCollection$LocalIndexerThread (IndexCollection.java:215) - new_document_parses/3296585.json: 1 docs added.
2022-12-15 19:12:56,778 DEBUG [pool-2-thread-6] index.IndexCollection$LocalIndexerThread (IndexCollection.jav

In [ ]:
tree = ET.parse('/content/topics2016.xml')
root = tree.getroot()

new_dict = {}

for child in root:
    for summary in child.iter('summary'):
        new_dict[child.attrib['number']] = summary.text

print(new_dict)

{'1': 'A 78 year old male presents with frequent stools and melena.', '2': 'An elderly female with past medical history of right hip arthroplasty presents after feeling a snap of her right leg and falling to the ground.', '3': 'A 75F found to be hypoglycemic with hypotension and bradycardia. She had  UA positive for klebsiella. She had a leukocytosis to 18 and a creatinine of 6.  Pt has blood cultures positive for group A streptococcus.  On the day of transfer her blood pressure dropped to the 60s.  She was anuric throughout the day, awake but drowsy.  This morning she had temp 96.3, respiratory rate 22, BP 102/26.', '4': 'An 87 yo woman with h/o osteoporosis, DM2, dementia, depression, and anxiety presents s/p fall with evidence of C2 fracture, chest pain, tachycardia, tachypnea, and low blood pressure.', '5': 'An 82 man with multiple chronic conditions and previous surgeries presents with 9 day history of productive cough, fever and dyspnea.', '6': 'A 94 year old female with hx recen

In [ ]:
dict_split1 = dict(list(new_dict.items())[len(new_dict)//2:])
dict_split2 = dict(list(new_dict.items())[:len(new_dict)//2])

In [ ]:
dict_split2

{'1': 'A 78 year old male presents with frequent stools and melena.',
 '2': 'An elderly female with past medical history of right hip arthroplasty presents after feeling a snap of her right leg and falling to the ground.',
 '3': 'A 75F found to be hypoglycemic with hypotension and bradycardia. She had  UA positive for klebsiella. She had a leukocytosis to 18 and a creatinine of 6.  Pt has blood cultures positive for group A streptococcus.  On the day of transfer her blood pressure dropped to the 60s.  She was anuric throughout the day, awake but drowsy.  This morning she had temp 96.3, respiratory rate 22, BP 102/26.',
 '4': 'An 87 yo woman with h/o osteoporosis, DM2, dementia, depression, and anxiety presents s/p fall with evidence of C2 fracture, chest pain, tachycardia, tachypnea, and low blood pressure.',
 '5': 'An 82 man with multiple chronic conditions and previous surgeries presents with 9 day history of productive cough, fever and dyspnea.',
 '6': 'A 94 year old female with hx 

In [ ]:
from pyserini.index import IndexReader

index_reader = IndexReader('indexes/new_index')

In [ ]:
index_reader.stats()

{'total_terms': 49262401,
 'documents': 16333,
 'non_empty_documents': 16333,
 'unique_terms': -1}

In [ ]:
len_C = index_reader.stats()['total_terms']

In [ ]:
def len_doc(d):
    doc_vector = index_reader.get_document_vector(d)

    len_d = 0
    for value in doc_vector.values():
      len_d += value
    
    return len_d

In [ ]:
def coll_freq(t):

  try:
    cf = index_reader.get_term_counts(t)[1]
    return cf

  except:
    return 0


In [ ]:
def term_freq(t, d):
    doc = index_reader.get_document_vector(d)
    
    if t in doc: tf = doc[t]
    else: tf = 0
    
    return tf

In [ ]:
def prob_t_Md(t, d, lambd):

    p_t_Md = (1-lambd)*(term_freq(t, d) / abs(len_doc(d))) + lambd*(coll_freq(t) / abs(len_C))

    return p_t_Md


In [ ]:
import math

def score_doc(q, d, lambd):

    p_q_Md_total = 0

    for term in q:

      p_q_Md_single = prob_t_Md(term, d, lambd)

      if p_q_Md_single == 0:
        continue

      else:
        p_q_Md_total += math.log(p_q_Md_single) 

    return p_q_Md_total
         

In [ ]:
from pyserini.search.lucene import LuceneSearcher

def write_run_file(queries, n_results, name):

    results = []
    searcher = LuceneSearcher("indexes/new_index")

    lambd = 0.1

    for qid, q in queries.items():

        hits = searcher.search(q, k=n_results)
        query = index_reader.analyze(q)

        for hit in hits:
          final_score = score_doc(query, hit.docid, lambd)
          results.append([qid, hit.docid, final_score])   

    check = set()
    for res in results:
        if ((res[0], res[1])) in check:
            raise Exception("Error: Duplicate query-doc is found", res[0], res[1])
        check.add((res[0], res[1]))

    # write results in a file
    output_str = "\n".join([l[0] + "\tQ0\t" + l[1] + "\t0\t" + str(l[2]) + "\tlm_jm" for l in results])
    open("{}_lm_jm.run".format(name), "w").write(output_str)

In [ ]:
write_run_file(dict_split1, 50, 'test_cds_split1')

In [ ]:
write_run_file(dict_split2, 50, 'test_cds_split2')